Required versions to reproduce results

*  Python 3.12.12
*  numpy: 2.0.2
*  pandas: 2.2.2
*  seaborn: 0.13.2
*  matplotlib: 3.10.0

In [1]:
# check python version
!python --version

Python 3.12.12


In [2]:
# import libraries and check versions

import pandas as pd
import numpy as np
print("numpy:", np.__version__)
print("pandas:", pd.__version__)

numpy: 2.0.2
pandas: 2.2.2


Exploratory Data Analytics Report generated previously.

Download the report from the project GitHub: https://github.com/jammaro14-sys/CIND820

# Prep data for analysis

In [3]:
# import dataset

df = pd.read_csv('https://data.ontario.ca/dataset/1f14addd-e4fc-4a07-9982-ad98db07ef86/resource/4cc07c1b-62ed-4ece-a2a4-d05d0f45081c/download/img-wage-rate-by-edu-age-sex-ft-pt-ca-on-2006-24.csv')

In [4]:
# some column names have leading space; identify to be renamed

for col in df.columns:
  if col.startswith(' '):
    print(f"Column '{col}' has a leading space.")

Column ' Men' has a leading space.
Column ' Women' has a leading space.


In [5]:
# Rename the ' Men' column to 'Men Wage' and ' Women' to 'Women Wage' to remove
# leading space and add clarity
df_renamed = df.rename(columns={' Men': 'Men_Wage', ' Women': 'Women_Wage'})

# Display the first few rows to show the renamed columns
display(df_renamed.head())

,YEAR,GEOGRAPHY,IMMIGRANT,TYPE OF WORK,WAGE RATE,EDUCATION,AGE GROUP,Both sexes,Men_Wage,Women_Wage
0,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",15 +,17.5,19.2,16.0
1,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 +,19.4,21.5,17.5
2,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 - 34,18.0,19.0,16.8
3,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 - 54,19.5,21.5,17.8
4,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 - 64,19.5,21.5,17.6


In [6]:
# check for NA values

df_renamed.isna().sum()

,0
YEAR,0
GEOGRAPHY,0
IMMIGRANT,0
TYPE OF WORK,0
WAGE RATE,0
EDUCATION,0
AGE GROUP,0
Both sexes,0
Men_Wage,0
Women_Wage,0


In [7]:
# check for NULL values

df_renamed.isnull().sum()


,0
YEAR,0
GEOGRAPHY,0
IMMIGRANT,0
TYPE OF WORK,0
WAGE RATE,0
EDUCATION,0
AGE GROUP,0
Both sexes,0
Men_Wage,0
Women_Wage,0


There are no NA or NULL values in the dataset. However, the Exploratory Data Analysis (EDA) Report identified 0's in the 'Men' and 'Women' variables, now renamed to "Men_Wage" and "Women_Wage".

'Men_Wage' has 6030 intances of "0.0", while 'Women_Wage' has 5236 instances.

According to the contextual document for this dataset, "Statistics Canada suppresses estimates below 1,500 - values shown as "0.0". Missing values shown as "-"." (Stastics Canada, 2025).

These "0.0" may not be true zeroes, but instead may represent an underrepresented group.

### Sensitivity Check: Impact of "0.0" Values on Mean Wage

To determine what we'll do with the "0.0", we'll compare the mean wage calculated in two ways:

1.  **Excluding "0.0":** Calculate mean, filtering out the "0.0" values

2.  **Treating "0.0" Values as True Zeros:** Calculate the mean with the "0.0" values

In [8]:
# create copy of df_renamed
# this will be the dataset which includes "0.0" as true zeroes

df_renamed_copy = df_renamed.copy()
df_renamed_copy.head()

,YEAR,GEOGRAPHY,IMMIGRANT,TYPE OF WORK,WAGE RATE,EDUCATION,AGE GROUP,Both sexes,Men_Wage,Women_Wage
0,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",15 +,17.5,19.2,16.0
1,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 +,19.4,21.5,17.5
2,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 - 34,18.0,19.0,16.8
3,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 - 54,19.5,21.5,17.8
4,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 - 64,19.5,21.5,17.6


In [9]:
# in def_renamed change "0.0" in dataset to NaN and flag as suppressed

for col in ['Men_Wage', 'Women_Wage']:
    # Create a new column to flag suppressed values for each gender
    df_renamed[f'{col}_is_suppressed'] = df_renamed[col].astype(str).str.strip() == "0.0"
    # Replace "0.0" strings with np.nan
    df_renamed[col] = df_renamed[col].replace('0.0', np.nan)

# Display the first few rows to show the new columns and replaced values
display(df_renamed.head())

,YEAR,GEOGRAPHY,IMMIGRANT,TYPE OF WORK,WAGE RATE,EDUCATION,AGE GROUP,Both sexes,Men_Wage,Women_Wage,Men_Wage_is_suppressed,Women_Wage_is_suppressed
0,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",15 +,17.5,19.2,16.0,False,False
1,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 +,19.4,21.5,17.5,False,False
2,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 - 34,18.0,19.0,16.8,False,False
3,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 - 54,19.5,21.5,17.8,False,False
4,2006,Canada,Total,All employees,Median hourly wage,"Total, all education levels",25 - 64,19.5,21.5,17.6,False,False


In [10]:
# calcuate mean Men_Wage and Women_Wage including the "0.0", using df_rename_copy

mean_men_wage_with_zeros = df_renamed_copy['Men_Wage'].mean()
mean_women_wage_with_zeros = df_renamed_copy['Women_Wage'].mean()

print(f"The mean men wage with zeroes included is {mean_men_wage_with_zeros}.")
print(f"The mean women wage with zeroes included is {mean_women_wage_with_zeros}.")

The mean men wage with zeroes included is 19.703499025341134.
The mean women wage with zeroes included is 17.312763157894736.


In [11]:
# calculate mean Men_wage and Women_wage, filtering out the supressed values in df_renamed

mean_men_wage_supressed_exluded = df_renamed['Men_Wage'].mean()
mean_women_wage_suppressed_excluded = df_renamed['Women_Wage'].mean()

print(f"The mean men wage with zeroes included is {mean_men_wage_supressed_exluded}.")
print(f"The mean women wage with zeroes included is {mean_women_wage_suppressed_excluded}.")

The mean men wage with zeroes included is 19.703499025341134.
The mean women wage with zeroes included is 17.312763157894736.


The calculated mean for both men and women, with and without the suppressed values are identical. From, this test, it seems to have no impact on analysis.

Let's do one more check.

Since the research questions will review wages over time, let's check the mean wages by year.

In [12]:
# Calculate average wage of Men by year, filtering out surpressed values

mean_men_wage_by_year = df_renamed[df_renamed['Men_Wage_is_suppressed'] == False].groupby('YEAR')['Men_Wage'].mean()
print(mean_men_wage_by_year)

YEAR
2006    18.973972
2007    19.468896
2008    20.380413
2009    20.325281
2010    20.368828
2011    20.780767
2012    21.162438
2013    21.317245
2014    21.724017
2015    22.244822
2016    22.733060
2017    23.371608
2018    23.667388
2019    24.310453
2020    25.773592
2021    26.636130
2022    27.558515
2023    28.620150
2024    29.224176
Name: Men_Wage, dtype: float64


In [13]:
# Calculate average wage of Men by year, including the "0.0"

mean_men_wage_by_year_with_zero = df_renamed_copy.groupby('YEAR')['Men_Wage'].mean()
print(mean_men_wage_by_year_with_zero)

YEAR
2006    16.233287
2007    16.575602
2008    17.342222
2009    17.568194
2010    17.304074
2011    17.807963
2012    17.762731
2013    18.198611
2014    18.676620
2015    18.794815
2016    19.228380
2017    19.855046
2018    20.259722
2019    20.888981
2020    21.191620
2021    22.048796
2022    24.049907
2023    24.724630
2024    25.855278
Name: Men_Wage, dtype: float64


In [14]:
merged_men_mean_wages = pd.DataFrame({
    'Mean Wage (Excluding Suppressed)': mean_men_wage_by_year,
    'Mean Wage (Including Suppressed)': mean_men_wage_by_year_with_zero
})

display(merged_men_mean_wages)

,Mean Wage (Excluding Suppressed),Mean Wage (Including Suppressed)
YEAR,,
2006,18.973972,16.233287
2007,19.468896,16.575602
2008,20.380413,17.342222
2009,20.325281,17.568194
2010,20.368828,17.304074
2011,20.780767,17.807963
2012,21.162438,17.762731
2013,21.317245,18.198611
2014,21.724017,18.676620


When calculating the mean men wage by year, you can see a difference in mean wages when the "0.0" are left in and when they are excluded. The calculated mean wages with the "0.0" are lower than when they are excluded.

Let's check if the same can be seen with women's mean wages.

In [15]:
# Calculate average wage of Women by year, filtering out surpressed values

mean_women_wage_by_year = df_renamed[df_renamed['Women_Wage_is_suppressed'] == False].groupby('YEAR')['Women_Wage'].mean()
print(mean_women_wage_by_year)

YEAR
2006    16.143964
2007    16.579383
2008    17.053254
2009    17.573637
2010    17.701858
2011    17.904984
2012    18.050131
2013    18.493941
2014    18.414650
2015    19.014933
2016    19.531774
2017    19.713665
2018    20.346429
2019    21.016693
2020    22.455863
2021    23.126461
2022    23.676728
2023    24.911737
2024    25.714714
Name: Women_Wage, dtype: float64


In [16]:
# Calculate average wage of Women by year, including the "0.0"

mean_women_wage_by_year_with_zero = df_renamed_copy.groupby('YEAR')['Women_Wage'].mean()
print(mean_women_wage_by_year_with_zero)

YEAR
2006    14.178287
2007    14.668148
2008    15.284769
2009    15.523380
2010    15.439954
2011    15.799491
2012    15.902500
2013    16.250694
2014    16.061667
2015    16.506019
2016    16.819028
2017    17.231204
2018    17.671250
2019    18.243657
2020    18.796389
2021    19.421944
2022    20.771944
2023    21.717037
2024    22.655139
Name: Women_Wage, dtype: float64


In [17]:
merged_women_mean_wages = pd.DataFrame({
    'Mean Wage (Excluding Suppressed)': mean_women_wage_by_year,
    'Mean Wage (Including Suppressed)': mean_women_wage_by_year_with_zero
})

display(merged_women_mean_wages)

,Mean Wage (Excluding Suppressed),Mean Wage (Including Suppressed)
YEAR,,
2006,16.143964,14.178287
2007,16.579383,14.668148
2008,17.053254,15.284769
2009,17.573637,15.523380
2010,17.701858,15.439954
2011,17.904984,15.799491
2012,18.050131,15.902500
2013,18.493941,16.250694
2014,18.414650,16.061667


There is also a difference in mean wages for women when the "0.0" are left in and when they are excluded. The calculated mean wages with the "0.0" are lower than when they are excluded.

This difference may be due to the "0.0" being evenly distributed across years, cancelling out proportionally.

The difference in mean wages by year seems to suggest that there could be an uneven distribution of zeroes by year. So if one yuear has many zeroes, the mean for that year would be lower, but excluding them would raise the mean.

Since the research questions will conduct analysis on wages over time, we will exclude the "0.0" from analysis.

Important to note that since suppression "0.0" is not random, there will be bias in the analysis. The analysis will solely be based on the population which is not supressed.